# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

Импортируем библиотеку pandas и библиотеку для стемминга, которая нам понадобится ближе к концу анализа.

In [1]:
import pandas as pd
from nltk.stem import SnowballStemmer

Открываем файл с днными и делаем обзор данных.

In [2]:
df = pd.read_csv('/datasets/data.csv')
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

Исходя из полученного результата, видим, что есть 12 столбцов, с ненулевыми значениями, имеются все 3 типа данных: целое, вещественное и строки. В 2х колонках есть пропуски.

### Шаг 2.1 Заполнение пропусков

Найдем и подсчитаем пропуски во всей таблице с данными

In [3]:
display(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

По 2174 пропущенных значений в 2х колонках: days_employed (стаж работы) и total_income (ежемесячный доход)

In [4]:
# Отношение пропусков к общему количеству строк
# display(df['total_income'].isna().sum() / df['total_income'].sum() * 100)

In [5]:
# Отношение пропусков к общему количеству строк
display(df.isna().mean())

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Рассчитали среднее значение пропусков к общемучислу строк в каждом столбце.
Из полученного результата видно, что доля пропусков в тех же двух столбцах составляет 10%.

In [6]:
# Определяем медианные значения по столбцам с пропусками и заполним ими пропуски в каждом столбце
k = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df['total_income'] = df['total_income'].fillna(df['total_income'].median())


# Проверка
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Все пропуски в двух столбцах заполнены медианным значением по соотвествующему столбцу. 
Причин, по которым появились пропуски, может быть несколько: 
+ Ошибка системы (сбой при выгрузке/загрузке данных)
+ Действительное отсутсиве данных (домохозяйка, студент - нет стажа, нет ежемесячного дохода)
+ Человеческий фактор (если какие-то из предложенных данных оцифровывались в ручную, то тот, кто оцифровывал, 
  мог ввести не верные значения или не ввести их вовсе)
  
В случае, когда данные содержат значения, которые могут резко выделятся на фоне с другими (например, стаж, ежемесячный доход), то статистически "медианный" показатель будет отражать более реальные цифры относительно большинства, чем среднее арифметическое. Поэтому было принято решение заполнить пропуски медианным значением.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [7]:
display(df)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


    Анализируя приведенные табличные данные, сразу бросаются в глаза значения в столбце с общим трудовым стажем в днях. Во-первых, большинство значений отрицательные. Отрицательный стаж не может быть. Возможно, при внесении данных что-то пошло не так. Отрицательные данные мы сохраним, но запишем их с положительным знаком. 
    
    Так же есть данные, значения которых во много превышают реальность. Например, 4ая строка - 340266 дней, это примерно 970 лет. Это тоже невозможно. Возможно, опять таки, допущена ошибка при вводе данных. Попробуем в таких случаях рассчитать максимально возможное количество трудовых дней за жизнь каждого заёмщика, по формуле: (возраст заёмщика - 16) * 247, где 16 - допустим, что заёмщик официально устроился работать с 16 лет, и 247 - это среднее количество рабочих дней в России за последние 5 лет (при 5 дневке). То есть, если значение будет превышать значение, рассчитанное по данной формуле, то мы его будем заменять на значение, рассчитанное по данной формуле.
    
    П.с. надеюсь ни один заемщик не обидется, что, возможно, я таким образом занижу их реальный трудовой стаж.
    И как будто бы логично: сначала поправить знак и слишком превосходящие значения, а потом уже заполнять пропуски медианным значением..

In [8]:
# Берём по модулю все значения столбца
df['days_employed'] = df['days_employed'].abs()
df['days_employed'] = df['days_employed'].astype('int')

def new_days_value(row):
    days = row['days_employed']
    years_old = row['dob_years']
    
    if days > (years_old - 16) * 365:
        days = (years_old - 16) * 365
    return days

df['days_employed'] = df.apply(new_days_value, axis=1)
l = df['days_employed'].median()

display(df)
display(k, l)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,13505,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,18615,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


-1203.369528770489

1795.0

Сравнив медианное значение до всех преобразований со столбцом "days_employed" (k = -1203.36...) и после (l = 1795.0), то видим, что медианное значение cтало, во-первых, положительным (что и планировалось получить), а во-вторых, более похожим на реальность

### Шаг 2.3. Изменение типов данных.

In [9]:
# Заменим тип столбца total_income с float на int:
df['total_income'] = df['total_income'].astype('int')

# И проверим сразу
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


    Успешно заменили вещественный тип данных в столбце total_income на целочисленный.

### Шаг 2.4. Удаление дубликатов.

    Рассмотрим столбцы, у которых тип данных - строка. Первым под проверку попадает столбец "education". Используем метод value_counts(), чтобы увидеть все уникальные значения в этом столбце.

In [10]:
display(df['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

    Если посчитать "ручками", то у нас должно получится только 5 уникальных значений, исходя из данной таблицы. Все остальные отличаются от этих 5 только регистром. Значит нужно все данные привести к какому-то одному общему регистру и параллельно проверим остальные столбцы со строковыми данными на наличие неявных дубликатов:

In [11]:
df['education'] = df['education'].str.lower()
# Проверка
display(df['education'].value_counts())

# Поиск неявных дубликатов в других столбцах
display(df['family_status'].value_counts())

display(df['gender'].value_counts())

display(df['income_type'].value_counts())

display(df['purpose'].value_counts())
# display(df)
# df.duplicated().sum()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Неявных дубликатов в других столбцах нет. Единственное, в столбце с гендером есть неопознаное значение XNA. С ним нужно что-то придумать и в столбце "цель" придется потом поиграться, чтобы немного объединить по смысловым группам.

In [12]:
# Определим индекс строки, где указано значени XNA и потом используем его, чтобы удалить данную строку
display(df[df['gender'] == 'XNA'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


In [13]:
# Удаляем строку с нужным индексом
df.drop(labels = [10701],axis = 0, inplace = True)

# Сбрасываем порядок индексов
df = pd.concat([df], ignore_index=0)
# Проверяем, что строка удалена
# df.info()

Было принято решение: принебречь строкой со значением гендера XNA и удалить её. Теперь, когда ручной поиск явных дубликатов окончен, то сделаем проверку на наличие неявных дубликатов.

In [14]:
df.duplicated().sum()

71

71 явных дубликатов после наших некоторых преобразований. Думаю можно ими тоже принибречь и удалить

In [15]:
# Удалим сначала явные дубликаты и "сбросим" индексы
df = df.drop_duplicates().reset_index(drop=True)
# Проверка
df.duplicated().sum()

0

Первоначально я сделал поиск явных дубликатов и сразу удалил их. Затем начал поиск неявных дубликатов вручную (с проверкой на регистр). Привёл к общему регистру, сделал проверку на наличие явных дубликатов и снова появились новые. Тогда пересмотрел тактику, решив, что это излишний повтор одних и тех же действий (дважды удалять явные дубликаты). Тогда сначала поискал все неявные дубликаты, разобрался с ними и только потом сделал поиск явных дубликатов и удалил их.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [16]:
# Создадим два датафрейма из исходного
df_education = df[['education_id', 'education']]
df_family = df[['family_status_id', 'family_status']]

# Проверка
# display(df_education)
display(df_family)

# Удаляем уже лишние столбцы из исходного датафрейма
df.drop(columns = ['education', 'family_status'],axis = 1, inplace=True)

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
...,...,...
21448,1,гражданский брак
21449,0,женат / замужем
21450,1,гражданский брак
21451,0,женат / замужем


### Шаг 2.6. Категоризация дохода.

Напишем собственную функцию для категоризации заёмщиков, в зависимости от их дохода. И создадим новый столбец с категорией доходности, использовав новую функцию.

In [17]:
def income_category(income):
    if income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    elif income > 1000000:
        return 'A'
    
df['total_income_category'] = df['total_income'].apply(income_category)

#Проверка
display(df['total_income_category'].value_counts())

C    16016
B     5040
D      350
A       25
E       22
Name: total_income_category, dtype: int64

Категории присвоены верно и успешно

### Шаг 2.7. Категоризация целей кредита.

Взглянем еще раз на уникальные значения в столбце "цель" займа. На оснвоании этих уникальных значений будет проще понять, как записать функцию с применением лемматизации.

In [18]:
display(df['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      620
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [19]:
from pymystem3 import Mystem
m = Mystem()

def purpose_lemmer(row):
    try:
        purpose_list = m.lemmatize(row['purpose'])
        if 'свадьба' in purpose_list:
            return 'проведение свадьбы'
        elif 'автомобиль' in purpose_list:
            return 'операции с автомобилем'
        elif 'образование' in purpose_list:
            return 'получение образования'
        elif 'недвижимость' in purpose_list or 'жилье' in purpose_list:
            return 'операции с недвижимостью'
    except:
        return 'нет информации'
    
df['purpose_category'] = df.apply(purpose_lemmer, axis=1)    

# Посмотрим на что больше всего люди хотят взять кредит
display(df['purpose_category'].value_counts())       


операции с недвижимостью    10810
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

Из вывода виден "топ целей" для взятия кредита. На первом месте это все, что связано с недвижимостью и жильем (покупка, ремонт..), на втором месте - все, что связано с автомобилем (так же покупка, ремонт..), третье - образование (получение высшего образования, дополнительного...), и на последнем месте это займ для проведения свадебных мероприятий.

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [20]:
#display(df)

#def check_children(row):
    #how_much = row['children']
    #if how_much < 0:
       # how_much = 0
       # return how_much
   # elif how_much == 20:
       # how_much = 2
      #  return how_much

#df[df['children'] < 0] = 0
#df[df['children'] == 20] = 2
#display(df[df['children'] >= 5].count())
#df.apply(check_children, axis=1)


df.loc[(df.children < 0), 'children'] = 0
df.loc[(df.children == 20), 'children'] = 2
#display(df['children'].value_counts())


df1 = df.groupby(['children']).agg({'debt':['sum', 'count']})
df1['percent'] = df1['debt']['sum'] / df1['debt']['count'] * 100

display(df1)
display(df1['percent'].sort_values())

debt          percent
           sum  count          
children                       
0         1064  14137  7.526349
1          444   4808  9.234609
2          202   2128  9.492481
3           27    330  8.181818
4            4     41  9.756098
5            0      9  0.000000

children
5    0.000000
0    7.526349
3    8.181818
1    9.234609
2    9.492481
4    9.756098
Name: percent, dtype: float64

##### Вывод 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

    В ходе изучения данных в столбце 'children', было обнаружено 47 значений равных "-1". Было принято решение таким значениям присоить значение 0, ссылаясь на то, что это возможно опять-таки "ошибка". Так же есть строки, где количество детей равно 20. Опираясь на то, что больше никаких других значений выше 5 в данных нет, то предположим, что это снова ошибка. И посчитаем, что люди с 2 детьми.

    Далее сгруппировали всех заемщиков по количеству детей, так же добавили столбцы с количеством невернувших кредит в срок и общего столбца заемщиков. И в отдельном столбце рассчитали отношение "невернувших" займ к общему количеству заемщиков в процентах. Из полученной таблицы видно, что кредит берут в основном те люди, у которых нет детей, на втором месте те, у кого 1 ребенок в семье, потом двое и так далее. То есть количество заёмщиков уменьшается пропрционально с увеличением количества детей в семье. 

    Если рассмтаривать по "успешности" кредитной истории, то люди с 5ью детьми в 100/100 случаев погашают кредит без задолжности, но при этом они почти и не берут кредит. Второе и третье место разделяют люди не имюещие детей и с 3 детьми соотвественно. Самый худший показатель у людей с 2мя детьми, практически каждый 6ой заемщик не возвращает кредит в срок.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Почему-то merge() у меня не работает. Юпитер пишет что ядро умерает, либо выдается ошибка, что не существует якобы датафрейма, хотя все ячейки до этого прекрасно работают. Перезагрузка тоже не помогла. Поэтому попробую ответить на вопросы, немного по-глупому пока что

In [21]:
# df_family_debt = df[['family_status_id', 'debt']]
# df2 = df_family.merge(df_family_debt, on='family_status_id',how='right')
# display(df2.head())


df_family['debt'] = df['debt']
df2 = df_family.groupby(['family_status']).agg({'debt':['sum', 'count']})
df2['percent'] = df2['debt']['sum'] / df2['debt']['count'] * 100
display(df2)
display(df2['percent'].sort_values())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


debt          percent
                       sum  count          
family_status                              
Не женат / не замужем  274   2810  9.750890
в разводе               85   1195  7.112971
вдовец / вдова          63    959  6.569343
гражданский брак       388   4150  9.349398
женат / замужем        931  12339  7.545182

family_status
вдовец / вдова           6.569343
в разводе                7.112971
женат / замужем          7.545182
гражданский брак         9.349398
Не женат / не замужем    9.750890
Name: percent, dtype: float64

##### Вывод 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

    Из полученных данных, видим, что заемщики вдовы и вдовцы имеют самую успешную кредитную историю 6,5%. Но и берут кредит они, в целом, в разы меньше, чем женатые и замужние. Чей показатель по задолженсти расположился на 3ьем месте - 7,5%. Самый худший показатель задолжности у не женатых и не замужних - каждый 10% практически выплачивает кредит не в срок.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [22]:
# df3 = df.groupby(['total_income_category']).agg({'debt':['sum', 'count']})
# df3['percent'] = df3['debt']['sum'] / df3['debt']['count'] * 100
# display(df3)
# display(df3['percent'].sort_values())

df3 = df.pivot_table(index='total_income_category', values = 'debt', aggfunc=['sum', 'count', 'mean'])
df3.columns = ['sum', 'count', 'mean']
#df3.sort_values(by='mean')
display(df3)
# 0–30000 — 'E';
# 30001–50000 — 'D';
# 50001–200000 — 'C';
# 200001–1000000 — 'B';
# 1000001 и выше — 'A'

,sum,count,mean
total_income_category,,,
A,2,25,0.080000
B,356,5040,0.070635
C,1360,16016,0.084915
D,21,350,0.060000
E,2,22,0.090909


##### Вывод 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

    Из таблицы видим, что люди из категории D (с доходом от 30к-50к) редко берут кредит и с большей долей погашают его в срок.
    Хуже всех погашают кредит в срок люди, чей доход составляет меньше 30к рублей, но при этом они и берут кредит крайне редко.
    Чаще всего кредит берут люди из категории С (доход 50к-200к) и при этом занимают 4е место по количеству не должников в топ5

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [23]:
df4 = df.groupby(['purpose_category']).agg({'debt':['sum', 'count']})
df4['percent'] = df4['debt']['sum'] / df4['debt']['count'] * 100
display(df4)
display(df4['percent'].sort_values())

debt          percent
                          sum  count          
purpose_category                              
операции с автомобилем    403   4306  9.359034
операции с недвижимостью  782  10810  7.234043
получение образования     370   4013  9.220035
проведение свадьбы        186   2324  8.003442

purpose_category
операции с недвижимостью    7.234043
проведение свадьбы          8.003442
получение образования       9.220035
операции с автомобилем      9.359034
Name: percent, dtype: float64

##### Вывод 4: Как разные цели кредита влияют на его возврат в срок?

    Из полученного топ4 видим, что чаще всего в срок кредит возвращают люди, которую берут его для покупки недвижимости, ремонта и так далее. И при этом такая категория людей чаще всего, в целом, и обращаются в банк за кредитом.
    Самой худшей кредитной историей обладают люди, которые берут кредит для операций с автомобилем (покупка, ремнот..)


## Общий вывод:

В ходе данного исследования можно сделать следующие выводы. Мы провели 4 сравнительных анализа о том, как влияет разная категория заемщиков на их успешную кредитную историю. Таким образом выяснили, что чаще всего погашают кредит в срок люди: 
 + многодетные заемщики и те, у кого нет детей вовсе (~0-7.5);
 + вдовы/ вдовцы или те, кто в разводе (~6,5-7,1%);
 + люди с ежемесячным доходом от 30к-50к рублей или люди с доходом 200к-1млн (~6.0-7.0%);
 + заещики, чья цель взятия кредита связана с недвижимостью или свадьбой (~7.2-8.0%).
Все оставшиеся категории в каждом пункте соотвественно имеют более низкие показатели.

Таким образом, можно составить примерно описать и тип клиента, которой с наибольшей вероятностью просрочет выплату кредита:
   - клиент, у кого 2е и больше детей, не состоит в браке, доход до 30к рублей и цель кредита связана с автомобилем.